In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_mcc_country = '''SELECT time_period_value, mcc, spend, destination_country
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value IN ("2024Q1","2024Q2","2024Q3","2024Q4")
and merchant_channel = 'Online'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and destination_country != 'UNITED KINGDOM'
and mcc != 'All'
GROUP BY mcc, time_period_value, spend, destination_country
ORDER BY time_period_value, spend DESC'''

mcc_country_df = bq.read_bq_table_sql(client, UK_mcc_country)

mcc_country_df['year'] = mcc_country_df['time_period_value'].str[:4]

mcc_country_df.head()

In [ ]:
# Filter for selected countries
selected_countries = [
    "REPUBLIC OF IRELAND", "UNITED STATES OF AMERICA", "NETHERLANDS",
    "REST OF EUROPE", "SPAIN", "FRANCE", "GERMANY", "REST OF ASIAPAC"
]
mcc_country_df = mcc_country_df[mcc_country_df['destination_country'].isin(selected_countries)]

# Calculate total spend per country
country_totals = mcc_country_df.groupby('destination_country')['spend'].sum().reset_index()
country_totals.columns = ['destination_country', 'total_spend']

# Merge and calculate percent spend
mcc_country_df = mcc_country_df.merge(country_totals, on='destination_country')
mcc_country_df['percent_spend'] = mcc_country_df['spend'] / mcc_country_df['total_spend'] * 100

# Get top 5 MCCs by percent spend per country
top_mccs = mcc_country_df.groupby('destination_country').apply(lambda x: x.nlargest(5, 'percent_spend')).reset_index(drop=True)

# Prepare data
top_mccs['label'] = top_mccs['mcc']
top_mccs['country'] = top_mccs['destination_country']
top_mccs = top_mccs.sort_values(by=['country', 'percent_spend'], ascending=[True, False]).reset_index(drop=True)
top_mccs['percent_norm'] = top_mccs['percent_spend'] / top_mccs['percent_spend'].max() * 5  # shorter bars

# Compute angles with spacing between countries
country_groups = top_mccs.groupby('country')
angles = []
angle_offset = 0
gap = 0.2  # gap between country groups in radians

for country, group in country_groups:
    n = len(group)
    angle_step = (2 * np.pi - gap * len(country_groups)) / len(top_mccs)
    group_angles = [angle_offset + i * angle_step for i in range(n)]
    angles.extend(group_angles)
    angle_offset = group_angles[-1] + gap

top_mccs['angle'] = angles

# Assign colors by country
top_mccs['color'] = top_mccs['country'].astype('category').cat.codes
cmap = plt.get_cmap("tab10")
colors = [cmap(i % 10) for i in top_mccs['color']]

# Plot
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'polar': True})
inner_radius = 5
bars = ax.bar(
    top_mccs['angle'],
    top_mccs['percent_norm'],
    width=0.25,
    bottom=inner_radius,
    color=colors,
    edgecolor='black',
    linewidth=0.5
)

# Add MCC labels
for idx, row in top_mccs.iterrows():
    angle = row['angle']
    radius = inner_radius + row['percent_norm'] + 0.5
    rotation = np.rad2deg(angle)
    alignment = 'left' if np.pi/2 <= angle <= 3*np.pi/2 else 'right'
    ax.text(
        angle,
        radius,
        row['label'],
        ha=alignment,
        va='center',
        rotation=rotation,
        rotation_mode='anchor',
        fontsize=8
    )

# Add country names in the center of each group
for country, group in country_groups:
    angle = np.mean(top_mccs[top_mccs['country'] == country]['angle'])
    ax.text(
        angle,
        inner_radius - 1,
        country,
        ha='center',
        va='center',
        fontsize=10,
        fontweight='bold',
        rotation=np.rad2deg(angle),
        rotation_mode='anchor'
    )

# Final adjustments
ax.set_yticklabels([])
ax.set_xticks([])
ax.set_facecolor('white')
fig.patch.set_facecolor('white')
plt.tight_layout()

# Save the figure
plt.savefig("circular_bar_plot_offset_bars.png", dpi=300)
plt.show()

